#### Creating Averages for H2S

**Import Functions and Data**

In [0]:
from pyspark.sql.functions import col, when, count, round, avg, percentile, to_date, max

silverConditions = spark.read.table("gasmonitordata.silver.monitorh2sandconditions")
silverInfo = spark.read.table("gasmonitordata.silver.monitorinformation")

In [0]:
avgH2S = silverConditions.alias('c').join(silverInfo.alias('i'), silverConditions.monitorNum == silverInfo.monitorNum). \
    withColumn('readingDate', to_date(col('c.timeStamp'))). \
    groupBy('c.monitorNum', 'i.groupNum', 'readingDate'). \
    agg(
        round(avg(col('c.h2sReading')), 2).alias('meanH2S'),
        round(percentile(col('c.h2sReading'), 0.75), 2).alias('p75_H2S'),
        round(max(col('c.h2sReading')), 2).alias('maxH2S'),
    ). \
    orderBy('c.monitorNum', 'i.groupNum', 'readingDate'). \
    select('c.monitorNum', 'i.groupNum', 'readingDate', 'meanH2S', 'p75_H2S', 'maxH2S')

In [0]:
avgH2S.write.mode('overwrite').saveAsTable('gasmonitordata.gold.avgH2S')